In [19]:
import robustness
import torch
from robustness.model_utils import make_and_restore_model
from robustness.datasets import ImageNet

In [20]:
ds = ImageNet('/home/mallet/Downloads/val')

In [25]:
model,_ = make_and_restore_model(arch = 'googlenet', dataset=ds, resume_path='/home/mallet/Downloads/GoogLeNet_R.pt.best.pt')

KeyError: 'googlenet'

In [22]:
for p in model.parameters():
        p.requires_grad = False
if torch.cuda.is_available():
    model.cuda()
model.eval()
def modells(img):
    sftmx, _ = model(img)
    return sftmx
    
    

In [23]:
import torchvision
import torchvision.transforms as transforms
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
def normalize_transform():
    return transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
def inverse_normalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor
    
val_transforms = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(),normalize_transform()])    
val_dataset = torchvision.datasets.ImageFolder('/home/mallet/Desktop/VanillaVsARobust/validationSample/',transform = val_transforms)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, num_workers=4)

In [24]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        outputs = modells(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 34 %


In [14]:
import sys
import os
sys.path.append('/home/mallet/Desktop/sam')
os.chdir('/home/mallet/Desktop/sam')
from sam.utils import load_madry_model

In [15]:
mdl = load_madry_model(arch='madry_alexnet', my_attacker=True)

My Attacker is True
Architecture madry_alexnet not implemented.
Exiting


SystemExit: 1